In [136]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
import pandas as pd
from geopy.distance import vincenty
%matplotlib inline

# Initialisation Data frame

In [137]:
dfinal=pd.DataFrame(columns=('BarLong','BarLat','Rayon','Weight','NbDays','Month','Year','Id'))
dfinal.loc[0] =[0,0,0,0,0,0,0,0]

# Lecture des données

In [138]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month

In [139]:
df = pd.read_csv('../data/aden.csv',sep=';')
df2=df[df["large_area"]!="G"]
df=df[df["large_area"]=="G"]

size = len(df)

In [140]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (with max distance)')
        plt.xlabel('sample index')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [141]:
df["mois (0)"]=df.apply(lambda row: monthCorrect(row['mois (0)']), axis=1)
df["date"]=df.apply(lambda row: str(row["Annee (0)"]) +"-"+ str(row["mois (0)"]), axis=1)

In [142]:
df["date"].unique()

array(['2010-10', '2010-01', '2010-02', '2010-03', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-11', '2010-04', '2010-12',
       '2008-02', '2008-03', '2008-04', '2008-05', '2008-07', '2008-08',
       '2008-09', '2008-10', '2008-11', '2008-12', '2008-06', '2009-01',
       '2009-02', '2009-03', '2009-04', '2009-05', '2009-07', '2009-12',
       '2009-06', '2009-08', '2009-09', '2009-10', '2009-11', '2011-01',
       '2011-04', '2011-09', '2011-02', '2011-05', '2011-07', '2011-08',
       '2011-10', '2011-03', '2011-11', '2011-06', '2011-12', '2012-02',
       '2012-12', '2012-03', '2012-04', '2012-01', '2012-06', '2012-05',
       '2012-07', '2013-03', '2013-06', '2013-05', '2013-07', '2013-12',
       '2014-01', '2014-04', '2014-02', '2014-03', '2014-08', '2014-12'], dtype=object)

In [143]:
def clusterise(date, max_d):
    #sous ensemble des données correspondant à la date choisie
    clustersId = []
    subDf = df[df["date"]==date]
    print subDf.index.values
    subDate = date.split("-")
    #dans le cas où le cluster ne contient qu'un seul élément :
    if(len(subDf) == 1):
        maxIdx=dfinal.index.values.max() 
        dfinal.loc[maxIdx+1] =[subDf["longitude"].values[0], subDf["latitude"].values[0],0,1,1,subDate[1],subDate[0],date+'_'+str(1)]
        df["id_cluster"].loc[subDf.index.values[0]]=date+'_'+str(1)
        return dfinal
    
    #construction du dataframe 2D
    X = subDf[["longitude","latitude"]].values
    #clusterisation
    Z = linkage(X, 'ward', metric = 'euclidean')
    #affichage du dendrogram

    #fancy_dendrogram(
    #    Z,
    #    truncate_mode='lastp',
    #    p=12,
    #    leaf_rotation=90.,
    #    leaf_font_size=12.,
    #    show_contracted=True,
    #    max_d=max_d,  # arret du dendrogram
    #)
    #plt.show()
    
    #récuperation des clusters
    from scipy.cluster.hierarchy import fcluster
    clusters = fcluster(Z, max_d, criterion='distance')
    print clusters
      
    for p in range(0,len(subDf.index.values)):
        df["id_cluster"].loc[subDf.index.values[p]]=date+'_'+str(clusters[p])
        #print subDf.index.values[clusters==1]
        #print subDf["jour (0)"][subDf.index.values[clusters==1]]
    
    #calcul du barycentre de chaque cluster
    centroidX = []
    centroidY = []
    for i in range(min(clusters),max(clusters)+1):
        x = np.mean(X[clusters==i,0])
        y = np.mean(X[clusters==i,1])
        point = (x,y)
        dist = []
        for j in range(0,len(X[clusters==i,0])):
            point2=(X[clusters==i,0][j], X[clusters==i,1][j])
            dist.append(vincenty(point, point2).kilometers)
        centroidX.append(x)
        centroidY.append(y)

        maxJ = max(subDf["jour (0)"][subDf.index.values[clusters==i]])
        minJ = min(subDf["jour (0)"][subDf.index.values[clusters==i]])
        
        nbDay = maxJ - minJ
        print nbDay
        
        maxIdx=dfinal.index.values.max() 
        dfinal.loc[maxIdx+1] =[x, y, max(dist),len(clusters[clusters==i]),nbDay,subDate[1], subDate[0],date+'_'+str(i)]

    #affichage des clusters
    #plt.scatter(X[:,0], X[:,1], c=clusters, cmap='prism')  # plot points with cluster dependent colors
    #plt.scatter(centroidX,centroidY,cmap='prism')
    #plt.title('Clusters and their centroid')
    #plt.xlabel('Longitude')
    #plt.ylabel('Latitude')
    #plt.show()


In [144]:
#df["date"].unique().len()+1
# add id_cluster
df["id_cluster"] = ""
for i in range(0,len(df["date"].unique())):
#for i in range(0,2):
    print df["date"].unique()[i]
    clusterise(df["date"].unique()[i], 4.5)

2010-10
[ 23  25 198 199 200 201]
[2 2 1 1 1 1]
25
2
2010-01
[ 42 146 147]
[1 1 1]
16
2010-02
[ 43  44 148]
[1 1 2]
2
0
2010-03
[ 45 149 150 151 152 153 154 155]
[1 1 1 1 3 1 2 2]
27
2
0
2010-05
[ 47  48 165 166 167 168 169]
[1 3 2 3 2 1 3]
17
11
25
2010-06
[ 49  50 170 171 172 173 174 175 176 177 178 179 180 181]
[1 1 2 2 2 2 1 2 2 2 2 2 2 2]
26
25
2010-07
[ 51 182 183 184 185]
[1 1 1 1 1]
17
2010-08
[ 52  53 186 187 188 189 190 191 192 193 194 195]
[1 1 1 2 3 2 2 2 1 1 1 1]
26
8
0
2010-09
[ 54 196 197]
[1 1 2]
2
0
2010-11
[ 55 202 203]
[1 1 2]
13
0
2010-04
[ 76 157 158 159 162 163 164]
[2 1 1 1 4 3 3]
3
0
0
0
2010-12
[204 205]
[1 1]
0
2008-02
[219 277]
[1 1]
0
2008-03
[220 278 279]
[1 1 1]
13
2008-04
[221 222 280 281 282 283]
[1 1 1 1 1 1]
27
2008-05
[223 224 284 285 286 287]
[1 1 1 1 1 1]
9
2008-07
[225 290 291 292]
[1 2 1 1]
5
0
2008-08
[226 227 228 229 230 231 293 294 295]
[1 2 1 1 2 2 1 1 1]
15
10
2008-09
[232 233 234 235 236 237 296 297 298 299 300 301 302 303 304 305 306 307
 3

In [145]:
dfinal.head()

,BarLong,BarLat,Rayon,Weight,NbDays,Month,Year,Id
0,0.000000,0.000000,0.000000,0,0,0,0,0
1,49.162986,13.191875,47.973162,4,25,10,2010,2010-10_1
2,55.191667,13.616667,128.608970,2,2,10,2010,2010-10_2
3,47.850833,12.711389,96.305994,3,16,01,2010,2010-01_1
4,47.908333,13.033333,93.623201,2,2,02,2010,2010-02_1


In [146]:
dfinal.to_csv("../data/aden_golfe_matrix_cluster.csv",sep=';', encoding='utf-8', index=False)

In [12]:
clusterise('2010-04', 4.5)

KeyError: 'date'

In [120]:
df2["id_cluster"]=""

frames = [df, df2]

result = pd.concat(frames)
print result.head()
result.to_csv("../data/aden.csv",sep=';', index=False)